# ASX Stock Predictor - Advanced Technical Analysis

This notebook demonstrates a comprehensive stock prediction system for ASX stocks using multiple technical analysis methods:

- **Fibonacci retracements and extensions**
- **Breakout pattern detection**
- **Reversal pattern detection**
- **Elliott Wave structure**
- **Fair Value Gaps**
- **Candlestick pattern recognition**
- **Heikin Ashi trend signals**
- **Renko trend direction**
- **Harmonic pattern detection**
- **Support and Resistance levels**

## Predictions
The system provides price estimates for:
- Tomorrow
- One week ahead
- One month ahead

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

# Import the predictor
from asx_stock_predictor import ASXStockPredictor

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Load Stock Data

Let's load data for an ASX stock. You can replace the ticker with any ASX stock symbol.

**Common ASX stocks:**
- BHP.AX - BHP Group
- CBA.AX - Commonwealth Bank
- NAB.AX - National Australia Bank
- WBC.AX - Westpac Banking
- ANZ.AX - ANZ Banking
- CSL.AX - CSL Limited
- WES.AX - Wesfarmers
- RIO.AX - Rio Tinto

In [ ]:
# Configuration
TICKER = "BHP.AX"  # Change this to your desired ASX stock
PERIOD = "1y"       # Data period: 1mo, 3mo, 6mo, 1y, 2y, 5y, max

# Download data
print(f"Downloading data for {TICKER}...")
df = yf.download(TICKER, period=PERIOD, progress=False)

# Display basic info
print(f"\nData loaded successfully!")
print(f"Date range: {df.index[0].date()} to {df.index[-1].date()}")
print(f"Total periods: {len(df)}")
print(f"\nLatest prices:")
print(df.tail())

## 2. Initialize the Predictor

In [ ]:
# Create predictor instance
predictor = ASXStockPredictor(df)
print(f"Predictor initialized successfully for {TICKER}")
print(f"Technical indicators calculated.")

## 3. Get Price Predictions

The system combines multiple forecasting methods:
- Linear Regression
- EMA Momentum Projection
- Mean Reversion (Bollinger Bands)
- RSI-based prediction
- Technical Pattern-based prediction

In [ ]:
# Get predictions
predictions = predictor.predict_prices()

# Display predictions
current = predictions['current']
tomorrow = predictions['tomorrow']
week = predictions['week']
month = predictions['month']

print(f"\n{'='*60}")
print(f"PRICE PREDICTIONS FOR {TICKER}")
print(f"{'='*60}")
print(f"\nCurrent Price: ${current['price']:.2f} (as of {current['date'].date()})")
print(f"\n" + "-"*60)

print(f"\n📈 TOMORROW'S FORECAST:")
print(f"   Predicted Price: ${tomorrow['prediction']:.2f}")
print(f"   Expected Change: ${tomorrow['change']:.2f} ({tomorrow['change_percent']:.2f}%)")
print(f"   Range: ${tomorrow['range_low']:.2f} - ${tomorrow['range_high']:.2f}")
print(f"   Confidence: {tomorrow['confidence']:.0f}%")

print(f"\n📊 ONE WEEK FORECAST:")
print(f"   Predicted Price: ${week['prediction']:.2f}")
print(f"   Expected Change: ${week['change']:.2f} ({week['change_percent']:.2f}%)")
print(f"   Range: ${week['range_low']:.2f} - ${week['range_high']:.2f}")
print(f"   Confidence: {week['confidence']:.0f}%")

print(f"\n📅 ONE MONTH FORECAST:")
print(f"   Predicted Price: ${month['prediction']:.2f}")
print(f"   Expected Change: ${month['change']:.2f} ({month['change_percent']:.2f}%)")
print(f"   Range: ${month['range_low']:.2f} - ${month['range_high']:.2f}")
print(f"   Confidence: {month['confidence']:.0f}%")
print(f"\n{'='*60}")

## 4. Visualize Predictions

In [ ]:
# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Price History with Predictions
ax1 = axes[0, 0]
recent_data = df.tail(100)
ax1.plot(recent_data.index, recent_data['Close'], label='Historical Price', linewidth=2)
ax1.plot(predictor.df.tail(100).index, predictor.df['EMA_9'].tail(100), 
         label='EMA 9', alpha=0.7, linestyle='--')
ax1.plot(predictor.df.tail(100).index, predictor.df['EMA_21'].tail(100), 
         label='EMA 21', alpha=0.7, linestyle='--')

# Add prediction points
last_date = df.index[-1]
future_dates = [last_date + timedelta(days=1), last_date + timedelta(days=7), last_date + timedelta(days=30)]
pred_prices = [tomorrow['prediction'], week['prediction'], month['prediction']]
ax1.scatter(future_dates, pred_prices, color='red', s=100, zorder=5, label='Predictions')

# Add range bands
ax1.fill_between([future_dates[0]], [tomorrow['range_low']], [tomorrow['range_high']], 
                  alpha=0.2, color='red')

ax1.set_title(f'{TICKER} - Price History and Predictions', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Prediction Comparison
ax2 = axes[0, 1]
timeframes = ['Tomorrow', 'Week', 'Month']
changes = [tomorrow['change_percent'], week['change_percent'], month['change_percent']]
colors = ['green' if c > 0 else 'red' for c in changes]
bars = ax2.bar(timeframes, changes, color=colors, alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_title('Expected Price Changes (%)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Change (%)')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, change in zip(bars, changes):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{change:.2f}%', ha='center', va='bottom' if height > 0 else 'top')

# Plot 3: Confidence Scores
ax3 = axes[1, 0]
confidences = [tomorrow['confidence'], week['confidence'], month['confidence']]
bars = ax3.bar(timeframes, confidences, color=['green', 'yellow', 'orange'], alpha=0.7)
ax3.set_title('Prediction Confidence Levels', fontsize=14, fontweight='bold')
ax3.set_ylabel('Confidence (%)')
ax3.set_ylim(0, 100)
ax3.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, conf in zip(bars, confidences):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             f'{conf:.0f}%', ha='center', va='bottom')

# Plot 4: Technical Indicators
ax4 = axes[1, 1]
indicators = ['RSI', 'ADX', 'Stoch K']
values = [
    predictor.df['RSI'].iloc[-1],
    predictor.df['ADX'].iloc[-1],
    predictor.df['Stoch_K'].iloc[-1]
]
bars = ax4.barh(indicators, values, color=['purple', 'blue', 'teal'], alpha=0.7)
ax4.set_title('Current Technical Indicators', fontsize=14, fontweight='bold')
ax4.set_xlabel('Value')
ax4.grid(True, alpha=0.3, axis='x')

# Add value labels
for bar, val in zip(bars, values):
    width = bar.get_width()
    ax4.text(width, bar.get_y() + bar.get_height()/2.,
             f'{val:.2f}', ha='left', va='center')

plt.tight_layout()
plt.show()

## 5. Detailed Technical Analysis

### 5.1 Fibonacci Levels

In [ ]:
fib_levels = predictor.calculate_fibonacci_levels()

print("\nFIBONACCI RETRACEMENT & EXTENSION LEVELS:")
print("="*50)
print(f"Swing High: ${fib_levels['swing_high']:.2f}")
print(f"Swing Low:  ${fib_levels['swing_low']:.2f}")
print("\nRetracement Levels:")
for level in ['fib_0.236', 'fib_0.382', 'fib_0.500', 'fib_0.618', 'fib_0.786']:
    print(f"  {level}: ${fib_levels[level]:.2f}")
print("\nExtension Levels:")
for level in ['fib_1.272', 'fib_1.618', 'fib_2.618']:
    print(f"  {level}: ${fib_levels[level]:.2f}")

# Visualize Fibonacci levels
plt.figure(figsize=(14, 7))
recent = df.tail(50)
plt.plot(recent.index, recent['Close'], linewidth=2, label='Price')

# Plot Fibonacci levels
for level_name, level_price in fib_levels.items():
    if 'fib' in level_name:
        plt.axhline(y=level_price, color='gray', linestyle='--', alpha=0.5, linewidth=1)
        plt.text(recent.index[-1], level_price, f'  {level_name}', 
                verticalalignment='center', fontsize=8)

plt.title(f'{TICKER} - Fibonacci Retracement & Extension Levels', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 5.2 Support and Resistance Levels

In [ ]:
sr_levels = predictor.calculate_support_resistance()

print("\nSUPPORT & RESISTANCE LEVELS:")
print("="*50)
print(f"Current Price: ${sr_levels['current_price']:.2f}")
print("\nResistance Levels (above current price):")
for i, r in enumerate(sr_levels['resistance'], 1):
    distance = ((r - sr_levels['current_price']) / sr_levels['current_price']) * 100
    print(f"  R{i}: ${r:.2f} (+{distance:.2f}%)")
print("\nSupport Levels (below current price):")
for i, s in enumerate(sr_levels['support'], 1):
    distance = ((sr_levels['current_price'] - s) / sr_levels['current_price']) * 100
    print(f"  S{i}: ${s:.2f} (-{distance:.2f}%)")

# Visualize S/R levels
plt.figure(figsize=(14, 7))
recent = df.tail(50)
plt.plot(recent.index, recent['Close'], linewidth=2, label='Price', color='blue')

# Plot resistance
for i, r in enumerate(sr_levels['resistance'], 1):
    plt.axhline(y=r, color='red', linestyle='--', alpha=0.7, linewidth=2)
    plt.text(recent.index[0], r, f'  R{i}', color='red', fontweight='bold')

# Plot support
for i, s in enumerate(sr_levels['support'], 1):
    plt.axhline(y=s, color='green', linestyle='--', alpha=0.7, linewidth=2)
    plt.text(recent.index[0], s, f'  S{i}', color='green', fontweight='bold')

plt.title(f'{TICKER} - Support & Resistance Levels', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 5.3 Candlestick Patterns

In [ ]:
candle_patterns = predictor.recognize_candlestick_patterns()

print("\nCANDLESTICK PATTERNS DETECTED:")
print("="*50)
if any(candle_patterns.values()):
    for pattern, detected in candle_patterns.items():
        if detected:
            signal = "🟢 BULLISH" if pattern in ['hammer', 'bullish_engulfing', 'morning_star'] else "🔴 BEARISH"
            print(f"  {pattern.upper().replace('_', ' ')}: {signal}")
else:
    print("  No significant candlestick patterns detected.")

### 5.4 Reversal Patterns

In [ ]:
reversal_patterns = predictor.detect_reversal_patterns()

print("\nREVERSAL PATTERNS DETECTED:")
print("="*50)
if any(reversal_patterns.values()):
    for pattern, detected in reversal_patterns.items():
        if detected:
            signal = "🟢 BULLISH" if pattern in ['double_bottom', 'inverse_head_shoulders', 'falling_wedge'] else "🔴 BEARISH"
            print(f"  {pattern.upper().replace('_', ' ')}: {signal}")
else:
    print("  No reversal patterns detected.")

### 5.5 Breakout Detection

In [ ]:
breakout = predictor.detect_breakout_patterns()

print("\nBREAKOUT ANALYSIS:")
print("="*50)
print(f"Consolidating: {'Yes' if breakout['is_consolidating'] else 'No'}")
print(f"Consolidation Range: ${breakout['consolidation_range'][0]:.2f} - ${breakout['consolidation_range'][1]:.2f}")
print(f"Breakout Up: {'🟢 YES' if breakout['breakout_up'] else 'No'}")
print(f"Breakout Down: {'🔴 YES' if breakout['breakout_down'] else 'No'}")
print(f"Volume Confirmed: {'✓' if breakout['volume_confirmed'] else '✗'}")
print(f"Breakout Strength: {breakout['breakout_strength']}")

### 5.6 Elliott Wave Analysis

In [ ]:
elliott = predictor.analyze_elliott_wave()

print("\nELLIOTT WAVE ANALYSIS:")
print("="*50)
print(f"Wave Trend: {elliott['wave_trend']}")
print(f"Current Phase: {elliott['current_phase']}")
print(f"Total Waves Detected: {elliott['wave_count']}")
print(f"Peaks: {elliott['peaks_count']}, Troughs: {elliott['troughs_count']}")

### 5.7 Fair Value Gaps

In [ ]:
fvg = predictor.detect_fair_value_gaps()

print("\nFAIR VALUE GAPS:")
print("="*50)
if fvg:
    for gap in fvg:
        print(f"\n{gap['type']}:")
        print(f"  Gap Range: ${gap['gap_low']:.2f} - ${gap['gap_high']:.2f}")
        print(f"  Gap Size: ${gap['gap_size']:.2f} ({gap['gap_percent']:.2f}%)")
else:
    print("  No significant fair value gaps detected.")

### 5.8 Heikin Ashi Trend

In [ ]:
ha_signals = predictor.get_heikin_ashi_signals()

print("\nHEIKIN ASHI TREND SIGNALS:")
print("="*50)
print(f"Trend: {ha_signals['trend']}")
print(f"Green Candles (last 5): {ha_signals['green_candles']}")
print(f"Red Candles (last 5): {ha_signals['red_candles']}")
if ha_signals['reversal']:
    print(f"Reversal Signal: {ha_signals['reversal']}")

### 5.9 Renko Trend

In [ ]:
renko = predictor.get_renko_trend()

print("\nRENKO TREND DIRECTION:")
print("="*50)
print(f"Trend: {renko['trend']}")
print(f"Trend Strength: {renko['strength']:.1f}%")
print(f"Up Bricks: {renko['up_bricks']}")
print(f"Down Bricks: {renko['down_bricks']}")
print(f"Total Bricks (last 10): {renko['total_bricks']}")

### 5.10 Harmonic Patterns

In [ ]:
harmonic = predictor.detect_harmonic_patterns()

print("\nHARMONIC PATTERNS:")
print("="*50)
print(f"Swing Points Found: {harmonic.get('swing_points_found', 0)}")
print(f"Patterns Detected: {harmonic['patterns_detected']}")
if harmonic['patterns']:
    for pattern in harmonic['patterns']:
        print(f"\n  {pattern['type']} Pattern ({pattern['direction']}):")
        print(f"    Completion Level: ${pattern['completion']:.2f}")
else:
    print("  No harmonic patterns detected.")

## 6. Comprehensive Analysis

Get all analyses in one comprehensive report:

In [ ]:
analysis = predictor.get_comprehensive_analysis()

print("\n" + "="*70)
print(f"COMPREHENSIVE ANALYSIS FOR {TICKER}")
print("="*70)

# Overall Signal
signal_color = {
    'Strong Buy': '🟢',
    'Buy': '🟢',
    'Neutral': '🟡',
    'Sell': '🔴',
    'Strong Sell': '🔴'
}

print(f"\n{signal_color.get(analysis['overall_signal'], '')} OVERALL SIGNAL: {analysis['overall_signal']}")
print(f"   Bullish Signals: {analysis['bullish_signals']}")
print(f"   Bearish Signals: {analysis['bearish_signals']}")

# Price Predictions Summary
print("\n" + "-"*70)
print("PRICE PREDICTIONS:")
print("-"*70)
current_price = analysis['predictions']['current']['price']
for timeframe in ['tomorrow', 'week', 'month']:
    pred = analysis['predictions'][timeframe]
    direction = "↑" if pred['change'] > 0 else "↓"
    print(f"\n{timeframe.upper()}:")
    print(f"  Price: ${pred['prediction']:.2f} {direction}")
    print(f"  Change: ${pred['change']:.2f} ({pred['change_percent']:.2f}%)")
    print(f"  Range: ${pred['range_low']:.2f} - ${pred['range_high']:.2f}")
    print(f"  Confidence: {pred['confidence']:.0f}%")

# Key Indicators
print("\n" + "-"*70)
print("KEY TECHNICAL INDICATORS:")
print("-"*70)
ind = analysis['indicators']
print(f"RSI: {ind['rsi']:.2f} - {'Oversold' if ind['rsi'] < 30 else 'Overbought' if ind['rsi'] > 70 else 'Neutral'}")
print(f"MACD: {ind['macd']:.4f} (Signal: {ind['macd_signal']:.4f})")
print(f"ADX: {ind['adx']:.2f} - {'Strong Trend' if ind['adx'] > 25 else 'Weak Trend'}")
print(f"Stochastic K: {ind['stoch_k']:.2f}")
print(f"ATR: ${ind['atr']:.2f}")

# Trend Analysis
print("\n" + "-"*70)
print("TREND ANALYSIS:")
print("-"*70)
print(f"Heikin Ashi: {analysis['heikin_ashi']['trend']}")
print(f"Renko: {analysis['renko']['trend']} (Strength: {analysis['renko']['strength']:.0f}%)")
print(f"Elliott Wave: {analysis['elliott_wave']['wave_trend']}")

# Pattern Detection
print("\n" + "-"*70)
print("PATTERNS DETECTED:")
print("-"*70)
active_candle = [k for k, v in analysis['candlestick_patterns'].items() if v]
active_reversal = [k for k, v in analysis['reversal_patterns'].items() if v]
if active_candle:
    print(f"Candlestick: {', '.join(p.replace('_', ' ').title() for p in active_candle)}")
if active_reversal:
    print(f"Reversal: {', '.join(p.replace('_', ' ').title() for p in active_reversal)}")
if analysis['harmonic_patterns']['patterns']:
    print(f"Harmonic: {', '.join(p['type'] for p in analysis['harmonic_patterns']['patterns'])}")
if not (active_candle or active_reversal or analysis['harmonic_patterns']['patterns']):
    print("No significant patterns detected.")

print("\n" + "="*70)

## 7. Save Analysis to File

In [ ]:
# Create a detailed report
report_filename = f"{TICKER}_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

with open(report_filename, 'w') as f:
    f.write(f"ASX STOCK PREDICTION REPORT\n")
    f.write(f"{'='*70}\n")
    f.write(f"Stock: {TICKER}\n")
    f.write(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Current Price: ${current_price:.2f}\n")
    f.write(f"\nOverall Signal: {analysis['overall_signal']}\n")
    f.write(f"Bullish Signals: {analysis['bullish_signals']}\n")
    f.write(f"Bearish Signals: {analysis['bearish_signals']}\n")
    f.write(f"\n{'='*70}\n")
    f.write(f"\nPRICE PREDICTIONS:\n")
    f.write(f"{'-'*70}\n")
    for timeframe in ['tomorrow', 'week', 'month']:
        pred = analysis['predictions'][timeframe]
        f.write(f"\n{timeframe.upper()}:\n")
        f.write(f"  Predicted: ${pred['prediction']:.2f}\n")
        f.write(f"  Change: ${pred['change']:.2f} ({pred['change_percent']:.2f}%)\n")
        f.write(f"  Range: ${pred['range_low']:.2f} - ${pred['range_high']:.2f}\n")
        f.write(f"  Confidence: {pred['confidence']:.0f}%\n")

print(f"\nAnalysis report saved to: {report_filename}")

## 8. Quick Analysis for Multiple Stocks

In [ ]:
# Analyze multiple ASX stocks
asx_stocks = ['BHP.AX', 'CBA.AX', 'CSL.AX', 'NAB.AX', 'WBC.AX']

results = []

for ticker in asx_stocks:
    try:
        print(f"\nAnalyzing {ticker}...")
        stock_data = yf.download(ticker, period="6mo", progress=False)
        if len(stock_data) < 50:
            print(f"  Skipping {ticker} - insufficient data")
            continue
        
        pred = ASXStockPredictor(stock_data)
        analysis = pred.get_comprehensive_analysis()
        
        results.append({
            'Ticker': ticker,
            'Current': analysis['predictions']['current']['price'],
            'Tomorrow': analysis['predictions']['tomorrow']['prediction'],
            'Change_1D_%': analysis['predictions']['tomorrow']['change_percent'],
            'Week': analysis['predictions']['week']['prediction'],
            'Change_1W_%': analysis['predictions']['week']['change_percent'],
            'Month': analysis['predictions']['month']['prediction'],
            'Change_1M_%': analysis['predictions']['month']['change_percent'],
            'Signal': analysis['overall_signal'],
            'RSI': analysis['indicators']['rsi'],
            'ADX': analysis['indicators']['adx']
        })
    except Exception as e:
        print(f"  Error analyzing {ticker}: {str(e)}")

# Create comparison DataFrame
comparison_df = pd.DataFrame(results)

if not comparison_df.empty:
    print("\n" + "="*100)
    print("MULTI-STOCK COMPARISON")
    print("="*100)
    print(comparison_df.to_string(index=False))
    
    # Visualize comparison
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Tomorrow's changes
    axes[0].barh(comparison_df['Ticker'], comparison_df['Change_1D_%'], 
                 color=['green' if x > 0 else 'red' for x in comparison_df['Change_1D_%']])
    axes[0].set_title('Tomorrow\'s Expected Change (%)', fontweight='bold')
    axes[0].set_xlabel('Change (%)')
    axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    axes[0].grid(True, alpha=0.3)
    
    # Week's changes
    axes[1].barh(comparison_df['Ticker'], comparison_df['Change_1W_%'],
                 color=['green' if x > 0 else 'red' for x in comparison_df['Change_1W_%']])
    axes[1].set_title('Week\'s Expected Change (%)', fontweight='bold')
    axes[1].set_xlabel('Change (%)')
    axes[1].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    axes[1].grid(True, alpha=0.3)
    
    # Month's changes
    axes[2].barh(comparison_df['Ticker'], comparison_df['Change_1M_%'],
                 color=['green' if x > 0 else 'red' for x in comparison_df['Change_1M_%']])
    axes[2].set_title('Month\'s Expected Change (%)', fontweight='bold')
    axes[2].set_xlabel('Change (%)')
    axes[2].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("\nNo results to display.")

## Disclaimer

**IMPORTANT:** This predictor is for educational and research purposes only. 

- Not financial advice
- Past performance does not guarantee future results
- Always conduct your own research before making investment decisions
- Consider consulting with a licensed financial advisor
- The stock market involves risk, and you may lose money

The predictions are based on technical analysis and historical patterns, which may not accurately predict future price movements.